# Modeling - 5

In [13]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.metrics import log_loss

from utils.utils import *
from utils.modeling import *
from features.ohe_dept import *
from features.base_features import *
from preprocess.pipe1 import PreprocessingV1

### Preprocessing Pipeline 1

In [14]:
# Import file
df = import_data("../data-ignore/train.csv")

In [15]:
# OHE the departments
ohe_dept_df, transformer = ohe_dept(df)

ohe_dept_groupby_df = ohe_dept_groupby(ohe_dept_df)
ohe_dept_groupby_df.reset_index(inplace=True)
ohe_dept_groupby_df

,VisitNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191344,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95671,191345,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Create a new feature indicating if scancount is negative
df['returns'] = df['ScanCount'].apply(lambda x: 1 if x < 0 else 0)

# Goupby VisitNumber
groupby_df = groupby_visitnumber(df)

groupby_df['weekend'] = groupby_df['weekday'].apply(lambda x: 1 if x in ['Saturday','Sunday'] else 0)
groupby_df['avg_scancount_per_upc'] = groupby_df['total_scancount'] / (groupby_df['num_unique_upc'] + 1)
groupby_df['avg_scancount_per_dept'] = groupby_df['total_scancount'] / (groupby_df['num_unique_dept'] + 1)

groupby_df

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,contains_return,weekend,avg_scancount_per_upc,avg_scancount_per_dept
0,5,999,Friday,1,-1.000000,-1,1,1,1,0,-0.500000,-0.500000
1,7,30,Friday,2,1.000000,2,2,2,0,0,0.666667,0.666667
2,8,26,Friday,20,1.217391,28,6,16,1,0,1.333333,4.000000
3,9,8,Friday,3,1.000000,3,2,3,0,0,0.750000,1.000000
4,10,8,Friday,3,1.000000,3,2,3,0,0,0.750000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5,0,1,1.125000,2.250000
95670,191344,22,Sunday,5,1.000000,5,2,3,0,1,0.833333,1.666667
95671,191345,39,Sunday,13,1.307692,17,8,12,0,1,1.214286,1.888889
95672,191346,39,Sunday,17,1.000000,17,8,16,0,1,0.944444,1.888889


In [17]:
# Join the 2 groupby'ed dataframes on the VisitNumber
df = pd.merge(groupby_df, ohe_dept_groupby_df, on="VisitNumber", how="inner")
df

,VisitNumber,triptype,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,contains_return,weekend,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,5,999,Friday,1,-1.000000,-1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,30,Friday,2,1.000000,2,2,2,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,26,Friday,20,1.217391,28,6,16,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,9,8,Friday,3,1.000000,3,2,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,8,Friday,3,1.000000,3,2,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191343,25,Sunday,7,1.285714,9,3,5,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191344,22,Sunday,5,1.000000,5,2,3,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95671,191345,39,Sunday,13,1.307692,17,8,12,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191346,39,Sunday,17,1.000000,17,8,16,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Pre-preprocess
X_train, X_test, y_train, y_test = pre_preprocess(df)

# Preprocessing
preprocessing = PreprocessingV1()
X_train_proc = preprocessing.fit_transform(X_train)
X_test_proc = preprocessing.transform(X_test)

X_train_proc

X_train: (81322, 79), y_train: (81322,), X_test: (14352, 79), y_test: (14352,)


,num__num_unique_upc,num__avg_scancount,num__total_scancount,num__num_unique_dept,num__num_unique_fileline,num__contains_return,num__weekend,num__avg_scancount_per_upc,num__avg_scancount_per_dept,num__DepartmentDescription_1-HR PHOTO,...,num__DepartmentDescription_TOYS,num__DepartmentDescription_WIRELESS,num__DepartmentDescription_nan,cat__weekday_Friday,cat__weekday_Monday,cat__weekday_Saturday,cat__weekday_Sunday,cat__weekday_Thursday,cat__weekday_Tuesday,cat__weekday_Wednesday
74630,-0.663062,-0.038488,-0.632608,-0.823057,-0.666973,-0.361107,1.345875,-0.610983,-0.713340,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
61468,-0.663062,-3.064435,-0.827308,-0.823057,-0.666973,2.769260,-0.743011,-2.527609,-1.559269,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10207,-0.424896,-0.038488,-0.437909,-0.473153,-0.393816,-0.361107,-0.743011,-0.131827,-0.290375,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,1.0,0.0,0.0,0.0,0.0,0.0
25328,0.765932,-0.038488,0.535589,1.976168,0.835391,-0.361107,1.345875,0.210428,-0.036596,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15352,0.408683,-0.038488,0.243540,0.576556,0.562234,-0.361107,-0.743011,0.173091,0.273578,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57115,0.051435,0.177651,0.048840,-0.123250,-0.120659,-0.361107,-0.743011,0.347329,0.555555,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,0.0,0.0,1.0,0.0
360,-0.663062,1.474485,-0.535258,-0.823057,-0.666973,-0.361107,-0.743011,0.347329,-0.290375,-0.039015,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
89473,-0.067648,-0.254627,-0.145859,-0.123250,-0.393816,2.769260,1.345875,0.073526,0.132590,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,1.0,0.0,0.0,0.0,0.0
51195,0.051435,-0.038488,-0.048510,0.226653,0.015920,-0.361107,1.345875,0.107751,0.047997,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
# Train on 3 models
results_df = train_model(X_train_proc, y_train, X_test_proc, y_test)
results_df


Running Logistic Regression...
Running Random Forest...
Running XGBoost...


,Model,Train Loss,Test Loss
0,LogReg,1.137860,1.126625
1,RForest,1.641706,1.626271
2,XGBoost,0.917123,0.878460


# Predict on Kaggle

In [20]:
# Import data
kaggle_test = pd.read_csv('../data-ignore/test.csv', dtype={'Upc': str})
kaggle_test.head()

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,72503389714,1,SHOES,3002.0
1,1,Friday,1707710732,1,DAIRY,1526.0
2,1,Friday,89470001026,1,DAIRY,1431.0
3,1,Friday,88491211470,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2840015224,1,DSD GROCERY,4408.0


In [21]:
# OHE the departments
kaggle_ohe_dept_df, transformer = ohe_dept(kaggle_test)

kaggle_ohe_dept_groupby_df = ohe_dept_groupby(kaggle_ohe_dept_df)
kaggle_ohe_dept_groupby_df.reset_index(inplace=True)
kaggle_ohe_dept_groupby_df

,VisitNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,DepartmentDescription_BEDDING,DepartmentDescription_BOOKS AND MAGAZINES,DepartmentDescription_BOYS WEAR,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191339,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95671,191340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Create a new feature indicating if scancount is negative
kaggle_test['returns'] = kaggle_test['ScanCount'].apply(lambda x: 1 if x < 0 else 0)

# Goupby VisitNumber
kaggle_groupby_df = groupby_visitnumber_kaggle(kaggle_test)

kaggle_groupby_df['weekend'] = kaggle_groupby_df['weekday'].apply(lambda x: 1 if x in ['Saturday','Sunday'] else 0)
kaggle_groupby_df['avg_scancount_per_upc'] = kaggle_groupby_df['total_scancount'] / (kaggle_groupby_df['num_unique_upc'] + 1)
kaggle_groupby_df['avg_scancount_per_dept'] = kaggle_groupby_df['total_scancount'] / (kaggle_groupby_df['num_unique_dept'] + 1)

kaggle_groupby_df

,VisitNumber,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,contains_return,weekend,avg_scancount_per_upc,avg_scancount_per_dept
0,1,Friday,4,1.000000,4,3,4,0,0,0.800000,1.00
1,2,Friday,4,1.000000,4,3,3,0,0,0.800000,1.00
2,3,Friday,1,0.000000,0,1,1,1,0,0.000000,0.00
3,4,Friday,1,1.000000,1,1,1,0,0,0.500000,0.50
4,6,Friday,1,0.000000,0,1,1,1,0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,Sunday,7,1.714286,12,3,6,0,1,1.500000,3.00
95670,191339,Sunday,5,1.200000,6,3,5,0,1,1.000000,1.50
95671,191340,Sunday,1,2.000000,2,1,1,0,1,1.000000,1.00
95672,191341,Sunday,10,1.200000,12,5,10,0,1,1.090909,2.00


In [23]:
# Join the 2 groupby'ed dataframes on the VisitNumber
kaggle_df = pd.merge(kaggle_groupby_df, kaggle_ohe_dept_groupby_df, on="VisitNumber", how="inner")
kaggle_df

,VisitNumber,weekday,num_unique_upc,avg_scancount,total_scancount,num_unique_dept,num_unique_fileline,contains_return,weekend,avg_scancount_per_upc,...,DepartmentDescription_SEASONAL,DepartmentDescription_SERVICE DELI,DepartmentDescription_SHEER HOSIERY,DepartmentDescription_SHOES,DepartmentDescription_SLEEPWEAR/FOUNDATIONS,DepartmentDescription_SPORTING GOODS,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,DepartmentDescription_nan
0,1,Friday,4,1.000000,4,3,4,0,0,0.800000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Friday,4,1.000000,4,3,3,0,0,0.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Friday,1,0.000000,0,1,1,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Friday,1,1.000000,1,1,1,0,0,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,Friday,1,0.000000,0,1,1,1,0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,Sunday,7,1.714286,12,3,6,0,1,1.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95670,191339,Sunday,5,1.200000,6,3,5,0,1,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95671,191340,Sunday,1,2.000000,2,1,1,0,1,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95672,191341,Sunday,10,1.200000,12,5,10,0,1,1.090909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# Save the VisitNumber
kg_index = kaggle_df['VisitNumber'].astype(str)
kg_index.head()

0    1
1    2
2    3
3    4
4    6
Name: VisitNumber, dtype: object

In [25]:
# Preprocessing
preprocessing = PreprocessingV1()
preprocessing.fit_transform(X_train)
kaggle_proc = preprocessing.transform(kaggle_df)

kaggle_proc

,num__num_unique_upc,num__avg_scancount,num__total_scancount,num__num_unique_dept,num__num_unique_fileline,num__contains_return,num__weekend,num__avg_scancount_per_upc,num__avg_scancount_per_dept,num__DepartmentDescription_1-HR PHOTO,...,num__DepartmentDescription_TOYS,num__DepartmentDescription_WIRELESS,num__DepartmentDescription_nan,cat__weekday_Friday,cat__weekday_Monday,cat__weekday_Saturday,cat__weekday_Sunday,cat__weekday_Thursday,cat__weekday_Tuesday,cat__weekday_Wednesday
0,-0.305814,-0.038488,-0.340559,-0.123250,-0.257237,-0.361107,-0.743011,-0.035996,-0.290375,-0.039015,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.305814,-0.038488,-0.340559,-0.123250,-0.393816,-0.361107,-0.743011,-0.035996,-0.290375,-0.039015,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.663062,-1.551462,-0.729958,-0.823057,-0.666973,2.769260,-0.743011,-1.569296,-1.136304,-0.039015,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.663062,-0.038488,-0.632608,-0.823057,-0.666973,-0.361107,-0.743011,-0.610983,-0.713340,-0.039015,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.663062,-1.551462,-0.729958,-0.823057,-0.666973,2.769260,-0.743011,-1.569296,-1.136304,-0.039015,...,-0.1359,-0.092219,-0.072906,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,0.051435,1.042207,0.438239,-0.123250,0.015920,-0.361107,1.345875,1.305642,1.401484,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95670,-0.186731,0.264106,-0.145859,-0.123250,-0.120659,-0.361107,1.345875,0.347329,0.132590,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95671,-0.663062,1.474485,-0.535258,-0.823057,-0.666973,-0.361107,1.345875,0.347329,-0.290375,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95672,0.408683,0.264106,0.438239,0.576556,0.562234,-0.361107,1.345875,0.521568,0.555555,-0.039015,...,-0.1359,-0.092219,-0.072906,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
# Train on full train dataset
full_X_train = pd.concat([X_train_proc, X_test_proc])
full_y_train = pd.concat([y_train, y_test])

# Retrain model
xgb_model = XGBClassifier()
xgb_model.fit(full_X_train, full_y_train)
train_proba = xgb_model.predict_proba(full_X_train)

# Check performance on train
train_loss = log_loss(full_y_train, train_proba)
print(f"Train loss: {train_loss:.4f}")

# Predict on kaggle test
kg_proba = xgb_model.predict_proba(kaggle_proc)
kg_proba

Train loss: 0.5591


array([[8.9799196e-06, 9.5097830e-06, 9.1856477e-05, ..., 5.5756317e-03,
        6.1659899e-04, 7.6663150e-03],
       [5.2943360e-05, 2.1007903e-05, 2.7729626e-04, ..., 5.4371506e-03,
        2.3967435e-04, 1.2737212e-02],
       [5.7555890e-06, 5.8785069e-08, 1.0113156e-06, ..., 2.6899920e-07,
        1.9075848e-07, 9.9880850e-01],
       ...,
       [7.8948651e-04, 1.2093182e-05, 1.4127101e-04, ..., 4.7893318e-05,
        2.8078392e-05, 5.2597310e-02],
       [5.2586165e-07, 1.1805174e-06, 5.7744387e-06, ..., 1.5603890e-02,
        3.0067107e-02, 7.4669762e-05],
       [3.3502937e-07, 1.0113543e-07, 2.3574735e-07, ..., 1.3836949e-03,
        1.8984255e-04, 1.1533063e-04]], shape=(95674, 38), dtype=float32)

In [27]:
# Import headers from sample_submission
sample_sub = pd.read_csv('../data-ignore/sample_submission.csv')
headers = sample_sub.columns.to_list()
headers

['VisitNumber',
 'TripType_3',
 'TripType_4',
 'TripType_5',
 'TripType_6',
 'TripType_7',
 'TripType_8',
 'TripType_9',
 'TripType_12',
 'TripType_14',
 'TripType_15',
 'TripType_18',
 'TripType_19',
 'TripType_20',
 'TripType_21',
 'TripType_22',
 'TripType_23',
 'TripType_24',
 'TripType_25',
 'TripType_26',
 'TripType_27',
 'TripType_28',
 'TripType_29',
 'TripType_30',
 'TripType_31',
 'TripType_32',
 'TripType_33',
 'TripType_34',
 'TripType_35',
 'TripType_36',
 'TripType_37',
 'TripType_38',
 'TripType_39',
 'TripType_40',
 'TripType_41',
 'TripType_42',
 'TripType_43',
 'TripType_44',
 'TripType_999']

In [28]:
# Create submission dataframe with predictions
submission = pd.DataFrame(np.round(kg_proba, 4), index=kg_index, columns=headers[1:])
submission.reset_index(inplace=True)
submission

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.0000,0.0,0.0001,0.0002,0.0072,0.0201,0.0162,0.0016,0.0,...,0.0002,0.0005,0.3504,0.0673,0.0002,0.0051,0.0058,0.0056,0.0006,0.0077
1,2,0.0001,0.0,0.0003,0.0002,0.0581,0.0210,0.0106,0.0035,0.0,...,0.0006,0.0007,0.0291,0.1024,0.0003,0.0017,0.0028,0.0054,0.0002,0.0127
2,3,0.0000,0.0,0.0000,0.0000,0.0001,0.0005,0.0005,0.0000,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9988
3,4,0.0001,0.0,0.0001,0.0003,0.0050,0.0474,0.8724,0.0000,0.0,...,0.0002,0.0001,0.0005,0.0003,0.0000,0.0001,0.0004,0.0000,0.0000,0.0278
4,6,0.0000,0.0,0.0000,0.0000,0.0001,0.0005,0.0005,0.0000,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95669,191338,0.0000,0.0,0.0000,0.0003,0.0006,0.0001,0.0000,0.0001,0.0,...,0.0000,0.0001,0.0026,0.0945,0.0016,0.0002,0.0007,0.0011,0.0009,0.0002
95670,191339,0.0000,0.0,0.0000,0.0002,0.0004,0.0004,0.0006,0.0000,0.0,...,0.0001,0.0001,0.0010,0.0116,0.0001,0.0020,0.0621,0.0091,0.0005,0.0018
95671,191340,0.0008,0.0,0.0001,0.0028,0.0046,0.6962,0.1733,0.0001,0.0,...,0.0005,0.0001,0.0005,0.0005,0.0001,0.0000,0.0103,0.0000,0.0000,0.0526
95672,191341,0.0000,0.0,0.0000,0.0000,0.0003,0.0000,0.0000,0.0001,0.0,...,0.0000,0.0001,0.0002,0.0776,0.0018,0.0060,0.0141,0.0156,0.0301,0.0001


In [29]:
# Save submission to csv
submission.to_csv('../data-ignore/submission7.csv', header=True, index=False)

In [30]:
# Kaggle score: private = 0.87181, public = 0.89295
# Kaggle place 373 / 1043 = 36%